In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical

In [2]:
train_dir = './train'
correct_dir = os.path.join(train_dir, 'correct/0-correct')
not_brake_stand = os.path.join(train_dir, 'fake/1-not-on-the-brake-stand')
from_screen = os.path.join(train_dir, 'fake/2-from-the-screen')
from_screen_photoshop = os.path.join(train_dir, 'fake/3-from-the-screen+photoshop')
photoshop = os.path.join(train_dir, 'fake/4-photoshop')

In [3]:
label_categories = {
    'correct': 0,
    'not_brake_stand': 1,
    'from_screen': 2,
    'from_screen_photoshop': 3,
    'photoshop': 4
}

data = []
labels = []

def classifyImages(dir, label):
    for image_file in os.listdir(dir):
        image_path = os.path.join(dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))
        data.append(image)
        labels.append(label)


classifyImages(correct_dir, 0)
classifyImages(not_brake_stand, 1)
classifyImages(from_screen, 2)
classifyImages(from_screen_photoshop, 3)
classifyImages(photoshop, 4)

images = np.array(data)
images = images / 255.0 
labels = np.array(labels)

labels = to_categorical(labels, num_classes=5)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

input_layer = Input(shape=(128, 128, 3)) 

# Convolutional layers
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = MaxPooling2D((2, 2))(conv3)
flatten = Flatten()(pool3)
dense1 = Dense(128, activation='relu')(flatten)
dropout = Dropout(0.5)(dense1)
output_layer = Dense(5, activation='softmax')(dropout)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856 

In [6]:
history = model.fit(X_train, y_train, epochs=6, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/6
165/165 [==============================] - 93s 545ms/step - loss: 0.6906 - accuracy: 0.7792 - val_loss: 0.4097 - val_accuracy: 0.8550
Epoch 2/6
165/165 [==============================] - 85s 515ms/step - loss: 0.3941 - accuracy: 0.8698 - val_loss: 0.3198 - val_accuracy: 0.8755
Epoch 3/6
165/165 [==============================] - 83s 505ms/step - loss: 0.3311 - accuracy: 0.8857 - val_loss: 0.3002 - val_accuracy: 0.8876
Epoch 4/6
165/165 [==============================] - 84s 508ms/step - loss: 0.2785 - accuracy: 0.9030 - val_loss: 0.2865 - val_accuracy: 0.9005
Epoch 5/6
165/165 [==============================] - 85s 516ms/step - loss: 0.2365 - accuracy: 0.9151 - val_loss: 0.2895 - val_accuracy: 0.8990
Epoch 6/6
165/165 [==============================] - 84s 512ms/step - loss: 0.2148 - accuracy: 0.9233 - val_loss: 0.2766 - val_accuracy: 0.9043


In [7]:
model.save('a7.h5')

C:\Users\zhulu\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
test_dir = './test2'
test_data = []
for file in os.listdir(test_dir):
    img_path = os.path.join(test_dir, file)
    try:
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128,128))
            test_data.append(img)
        else:
            print(f"Skipping {file}")
    except e:
        print(e)
        
test_data = np.array(test_data) / 255.0




In [29]:
predict = model.predict(test_data)

1/1 [==============================] - 0s 62ms/step


In [30]:
predicted_labels = np.argmax(predict, axis=1)


In [32]:
import os
import shutil
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('a7.h5')

test_dir = './test2'

test_data = []

def preprocess_test_images(dir):
    for image_file in os.listdir(dir):
        image_path = os.path.join(dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))
        image = image / 255.0
        test_data.append((image_file, image))

preprocess_test_images(test_dir)

test_filenames, test_images = zip(*test_data)

test_images = np.array(test_images)

predictions = model.predict(test_images)

predicted_classes = np.argmax(predictions, axis=1)

root_dir = './testtg'
os.makedirs(root_dir, exist_ok=True)

for class_label in range(5):
    class_dir = os.path.join(root_dir, f'class_{class_label}')
    os.makedirs(class_dir, exist_ok=True)

low_prob_dir = os.path.join(root_dir, 'low_probability')
os.makedirs(low_prob_dir, exist_ok=True)

for i, file in enumerate(test_filenames):
    class_label = predicted_classes[i]
    source_path = os.path.join(test_dir, file)
    
    if predictions[i][class_label] < 0.55:
        target_dir = low_prob_dir
    else:
        target_dir = os.path.join(root_dir, f'class_{class_label}')
    
    target_path = os.path.join(target_dir, file)

    shutil.copy(source_path, target_path)

    img = cv2.imread(target_path)
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = f'Class: {class_label}'
    org = (10, 30) 
    font_scale = 1
    color = (255, 255, 255)
    thickness = 2
    img = cv2.putText(img, text, org, font, font_scale, color, thickness)

    cv2.imwrite(target_path, img)

print("Файлы перенесены и класс отображен на изображениях.")


1/1 [==============================] - 0s 217ms/step
Файлы перенесены и класс отображен на изображениях.


In [33]:
test_images = [item[1] for item in test_data]

test_images = np.array(test_images)

predict = model.predict(test_images)
predicted_labels = np.argmax(predict, axis=1)

for i, file in enumerate(os.listdir(test_dir)):
    print(f"File: {file}, Predicted Label: {predicted_labels[i]}")
    print("Class Probabilities:")
    
    max_prob_index = np.argmax(predict[i])
    for class_label, probability in enumerate(predict[i]):
        print(f"Class {class_label}: {probability:.4f}")
    
    if max_prob_index == predicted_labels[i] and predict[i][max_prob_index] < 0.55:
        print("May have a mistake")


1/1 [==============================] - 0s 56ms/step
File: 23344641.jpeg, Predicted Label: 1
Class Probabilities:
Class 0: 0.2300
Class 1: 0.3727
Class 2: 0.2749
Class 3: 0.0076
Class 4: 0.1148
May have a mistake
File: 23389951.jpeg, Predicted Label: 3
Class Probabilities:
Class 0: 0.1181
Class 1: 0.0050
Class 2: 0.0553
Class 3: 0.8031
Class 4: 0.0184
File: 23390645.jpeg, Predicted Label: 0
Class Probabilities:
Class 0: 0.8733
Class 1: 0.1154
Class 2: 0.0026
Class 3: 0.0005
Class 4: 0.0081
File: 23516653.jpeg, Predicted Label: 2
Class Probabilities:
Class 0: 0.0929
Class 1: 0.0141
Class 2: 0.6092
Class 3: 0.0022
Class 4: 0.2817
File: asd.jpeg, Predicted Label: 0
Class Probabilities:
Class 0: 0.9306
Class 1: 0.0306
Class 2: 0.0007
Class 3: 0.0002
Class 4: 0.0381


In [13]:
import csv

csv_file = 'etovse6.csv'
csv_data = []

for file, predicted_label in zip(os.listdir(test_dir), predicted_labels):
    if not file.startswith('.ipynb_checkpoints'):
        if predicted_label > 1:
            predicted_label = 1
        file_name, file_extension = os.path.splitext(file)
        csv_data.append((file_name.replace('.jpeg', ''), predicted_label))

with open (csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['file_index', 'class'])
    writer.writerows(csv_data)


In [26]:
import cv2
import numpy as np

def load_and_predict_image(image_name, model):
    image_path = os.path.join(test_dir, image_name)
    
    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))
        image = image / 255.0
        image = np.expand_dims(image, axis=0)
        probabilities = model.predict(image)
        
        print(f"Probabilities for Image: {image_name}")
        for class_label, probability in enumerate(probabilities[0]):
            print(f"Class {class_label}: {probability:.4f}")
    else:
        print(f"Image '{image_name}' not found in the test data.")

image_name_to_check = "235116653.jpeg"
load_and_predict_image(image_name_to_check, model)


Image '235116653.jpeg' not found in the test data.
